In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd 
import numpy as np
import time
from sentence_transformers import SentenceTransformer,util
import tensorflow
import pickle

In [8]:
from fobokiller.cosim import load_embedding , compute_sim_df ,model

In [13]:
embedding = load_embedding()

2021-11-26 15:28:39.804607: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [132]:
df = compute_sim_df('I want to eat a pizza in a cosy place',np.array(embedding), 2000)
df['ratio']=df['review_filtered']/df['review_all']
df

,alias,date,rate,review_sentences_s,sim_s,rate_filtered,review_filtered,review_clean_r,sim_r,rate_all,review_all,ratio
0,pizza-julia-paris,2015-08-06,5,perfect place for some good pizza -- fresh ing...,0.663496,4.620253,45,perfect place for some good pizza -- fresh ing...,0.503197,4.538860,60,0.750000
1,firmine-paris,2018-08-16,5,must place to eat if you love pizza,0.658819,4.139241,67,great pizzas! we stopped here after we went on...,0.502106,3.903250,372,0.180108
2,falstaff-paris-2,2011-04-24,1,"if you want pizza, go across the street to the...",0.655241,1.500000,2,i just arrived in paris and was starved! the f...,0.560329,2.010753,10,0.200000
3,swann-et-vincent-paris-2,2015-05-07,4,a real comfortable italian restaurant,0.652533,4.000000,1,the place is really nice and the food is great...,0.652533,4.085714,4,0.250000
4,carmine-café-paris-3,2017-10-20,4,pizzas somewhere,0.637849,4.315789,15,pizzas somewhere . out of . i would change the...,0.501570,4.003279,80,0.187500
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,gambino-paris-2,2016-07-05,5,that's a very nice restaurant with a large cho...,0.456179,4.025000,35,my boyfriend and i split a pizza here. we just...,0.503422,4.227758,168,0.208333
1996,grazie-paris,2017-06-05,5,the pizza was delicious: thin crust with ampl...,0.456166,4.166667,19,"i live in brooklyn, new york so my pizza stand...",0.515450,3.889381,33,0.575758
1997,ober-mamma-paris,2016-10-13,5,"loved this place, planning on going back for ...",0.456149,4.733333,28,just so good. show up an hour early. put your ...,0.498792,4.403251,131,0.213740
1998,café-central-paris-5,2016-09-23,5,that was the best pizza we had in france,0.456133,4.714286,5,nice spot to enjoy some drinks. open late. bew...,0.469414,3.845258,89,0.056180


In [133]:
df['ratio_x_sim'] = df['ratio']*df['sim_r']
df['ponderation_review'] = df['ratio']*df['sim_r']*df['review_all']
df['ponderation_rate_review'] = df['ratio']*df['sim_r']*df.rate_all/df.review_all

In [134]:
df.groupby('alias').sum().sort_values(by = 'ponderation_rate_review').index

Index(['le-champ-de-mars-paris-4', 'café-de-flore-paris-3',
       'shakespeare-and-company-paris', 'candelaria-paris-2', 'angelina-paris',
       'le-relais-de-l-entrecôte-paris-3', 'la-droguerie-du-marais-paris',
       'bateaux-parisiens-paris', 'le-relais-de-l-entrecôte-paris-4',
       'le-bosquet-paris',
       ...
       'pizza-cesar-paris-2', 'louie-louie-paris', 'alice-pizza-paris-3',
       'pietro-paris-3', 'franco-et-giacomo-paris-2', 'pizza-di-loretta-paris',
       'pizza-julia-paris', 'il-brigante-paris', 'popine-paris',
       'pezzo-paris'],
      dtype='object', name='alias', length=598)

In [135]:
ponderation_rate_review = df.groupby('alias').sum().sort_values(by = 'ponderation_rate_review', 
                                      ascending=False)[:20].index

ponderation_review = df.groupby('alias').sum().sort_values(by = 'ponderation_review', 
                                      ascending=False)[:20].index


ratio = df.groupby('alias').sum().sort_values(by = 'ratio', 
                                      ascending=False)[:20].index
sim = df.groupby('alias').sum().sort_values(by = 'sim_r', 
                                      ascending=False)[:20].index
                                                                              

In [136]:
pd.DataFrame([sim,ratio,ponderation_rate, ponderation_review, ponderation_rate_review ], 
             index = ["sim","ratio", "ponderation_review", "ponderation_rate_review"]).T

ValueError: Shape of passed values is (5, 20), indices imply (4, 20)